In [35]:
from rdflib import Graph
import sys
sys.path.insert(0, '..')
from utils import query_to_df
import csv
import os
from pathlib import Path
import pyshacl
from brick_tq_shacl.topquadrant_shacl import infer
from buildingmotif.namespaces import * 
from pprint import pprint
import pandas as pd

In [13]:
gs = Graph(store = 'Oxigraph')
gs.parse('s223-example.ttl', format='ttl')
s223 = Graph(store = 'Oxigraph')
# s223.parse('s223-example.ttl')
s223.parse('../../223standard/models/core.ttl')
s223.parse('../../223standard/models/equipment.ttl')
# s223.parse('../../223standard/vocab/enumeration.ttl')
# s223.serialize('s223-example-parsed.ttl', format = 'ttl')
# s223.serialize('s223-relevant-ontology.ttl', format = 'ttl')

gb = Graph(store = 'Oxigraph')
gb.parse('brick-example-datagraph.ttl')
# gb_rdflib = Graph()
# gb_rdflib.parse('brick-example.ttl')
brick = Graph(store = 'Oxigraph')
brick.parse('../ontologies/Brick144.ttl')
gb_inferred = infer(gb, brick)

In [14]:
s223_inferred = infer(gs, s223)

In [15]:
# appears inference worked
for triple in gb_inferred:
    s,p,o = triple
    if 'AHU' in str(o):
        for p_all,o_all in gb_inferred.predicate_objects(s):
            if A == p_all:
                print(s,p_all,o_all)

urn:example#multiple-zone-ahu_name_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Air_Handler_Unit
urn:example#multiple-zone-ahu_name_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Air_Handling_Unit
urn:example#multiple-zone-ahu_name_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#AHU
urn:example#multiple-zone-ahu_name_0 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Air_Handler_Unit
urn:example#multiple-zone-ahu_name_0 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Air_Handling_Unit
urn:example#multiple-zone-ahu_name_0 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#AHU
urn:example#multiple-zone-ahu_name_2 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Air_Handler_Unit
urn:example#multiple-zone-ahu_name_2 http://www.w3.

In [ ]:
def read_queries_from_csv(csv_path):
    """
    Read queries from a CSV file.
    
    Args:
        csv_path: Path to the CSV file
        
    Returns:
        List of dictionaries containing query information
    """
    queries = []
    
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            query_info = {
                'query_id': row.get('query_id', ''),
                'question': row.get('question', ''),
                'generated_sparql': row.get('generated_sparql', ''),
                'model': row.get('model', ''),
                'source_file': os.path.basename(csv_path)
            }
            queries.append(query_info)
    
    return queries

# Directory containing the CSV files
results_dir = Path('react_kgqa_results')

# Find all CSV files
csv_files = list(results_dir.glob('*.csv'))

# print(f"Found {len(csv_files)} CSV files in {results_dir}\n")

all_queries = []
all_dfs = []
summary = []
# Process each CSV file
for csv_file in csv_files:
    print('='*80)
    print('Displaying File', csv_file.name)
    queries = read_queries_from_csv(csv_file)
    all_queries.extend(queries)
    g_i = gs if '_s223' in csv_file.name else gb
    for query in queries:
        print(query['question'])
        print(query['generated_sparql'])
        result = [csv_file.name, query['question'], False]
        try:
            df = query_to_df(query['generated_sparql'], g_i)
            all_dfs.append(df)
            result = [csv_file.name, query['question'], len(df) > 0]
            if df.empty:
                print('No results')
            else:
                print('Results found')
                for c in df.columns:
                    print(df[c].value_counts())
                    print('\n')
                display(df)
        except Exception as e:
            print('Error')
            print(e)
        finally:
            summary.append(result)

Displaying File ReAct(w100)_s223.csv
I'm too cold, find the components most likely at fault.
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?component ?label WHERE {
  # Restrict to physical components (devices, equipment, points, or s223:Component)
  ?component a ?type .
  FILTER(
    ?type IN (
      s223:Component,
      s223:QuantifiableObservableProperty,
      s223:QuantifiableActuatableProperty
    )
  )
  # Look for components involved in temperature control
  OPTIONAL { ?component rdfs:label ?label }
  FILTER (
    (BOUND(?label) && (
      CONTAINS(LCASE(STR(?label)), "damper") ||
      CONTAINS(LCASE(STR(?label)), "reheat") ||
      CONTAINS(LCASE(STR(?label)), "coil") ||
      CONTAINS(LCASE(STR(?label)), "heater") ||
      CONTAINS(LCASE(STR(

,component,label
0,urn:example#vav-with-reheat_name-ztemp_3_9,QuantifiableObservableProperty
1,urn:example#vav-with-reheat_name-ztemp_3_8,QuantifiableObservableProperty
2,urn:example#vav-with-reheat_name-ztemp_3_7,QuantifiableObservableProperty
3,urn:example#vav-with-reheat_name-ztemp_3_6,QuantifiableObservableProperty
4,urn:example#vav-with-reheat_name-ztemp_3_5,QuantifiableObservableProperty
...,...,...
251,urn:example#multiple-zone-ahu_oa_temp_0,QuantifiableObservableProperty
252,urn:example#multiple-zone-ahu_ma_temp_3,QuantifiableObservableProperty
253,urn:example#multiple-zone-ahu_ma_temp_2,QuantifiableObservableProperty
254,urn:example#multiple-zone-ahu_ma_temp_1,QuantifiableObservableProperty


What are all the instances of valve commands for cooling only VAVs?
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX s223: <http://data.ashrae.org/standard223#>

SELECT ?vav ?vav_label ?valve_command ?valve_command_label WHERE {
  # Find all VAVs (assuming a generic VAV class)
  ?vav rdf:type s223:VAV .
  OPTIONAL { ?vav rdfs:label ?vav_label }
  
  # Filter for cooling-only VAVs via label or property (if available)
  FILTER(
    (BOUND(?vav_label) && CONTAINS(LCASE(?vav_label), "cooling-only"))
    # Add additional FILTERs here if a property or tag for cooling-only exists, e.g.:
    # || EXISTS { ?vav s223:hasCoolingOnly true }
  )
  
  # Find associated valve command points via possible relationships
  {
    ?vav s223:hasPoint ?valve_command .
  } UNION {
    ?vav s223:hasValveCommand ?valve_command .
  } UNION {
    ?vav s223:hasProperty ?valve_command .
  }
  
  # Valve command type (if available)
  ?valve_command

,ahu,ahu_label,component,component_label
0,urn:example#multiple-zone-ahu_name_3,AirHandlingUnit,urn:example#vav-cooling-only_zone_0_0,DomainSpace
1,urn:example#multiple-zone-ahu_name_3,AirHandlingUnit,urn:example#vav-cooling-only_zone_0_1,DomainSpace
2,urn:example#multiple-zone-ahu_name_3,AirHandlingUnit,urn:example#vav-cooling-only_zone_0_10,DomainSpace
3,urn:example#multiple-zone-ahu_name_3,AirHandlingUnit,urn:example#vav-cooling-only_zone_0_11,DomainSpace
4,urn:example#multiple-zone-ahu_name_3,AirHandlingUnit,urn:example#vav-cooling-only_zone_0_12,DomainSpace
...,...,...,...,...
6107,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_ra_damper_3,Damper
6108,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#ahu_0con,Connection
6109,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#ahu_1con,Connection
6110,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#ahu_2con,Connection


What are the discharge air temperature sensors for all VAVs and their associated AHUs?
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>

SELECT ?vav ?vav_label ?sensor ?sensor_label ?ahu ?ahu_label WHERE {
  # Find VAVs by class or subclass
  ?vav rdf:type ?vav_class .
  ?vav_class rdfs:label ?vav_class_label .
  FILTER(REGEX(LCASE(STR(?vav_class_label)), "vav"))
  OPTIONAL { ?vav rdfs:label ?vav_label }

  # Find points or sensors associated with the VAV
  {
    ?vav ?point_pred ?sensor .
    FILTER(?point_pred IN (s223:hasPoint, s223:hasProperty, s223:hasSensor))
  } UNION {
    ?vav s223:hasProperty ?point .
    ?point rdf:type ?sensor_class .
    ?sensor_class rdfs:label ?sensor_class_label .
    FILTER(REGEX(LCASE(STR(?sensor_class_label)), "discharge.*air.*temperature

...

I'm too cold, find the components most likely at fault.
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# Replace this with the specific zone or room URI if known, e.g., :vav-cooling-only_zone_1_1
# For demonstration, we select all HVAC_Zone instances as possible affected zones
SELECT DISTINCT ?zone ?zone_label ?component ?component_label WHERE {
  ?zone a brick:HVAC_Zone .
  OPTIONAL { ?zone rdfs:label ?zone_label }

  # Find VAVs or terminal units that serve the zone
  ?vav (brick:feeds|^brick:feeds)* ?zone .
  ?vav a brick:VAV .

  # Find heating-related components that are part of the VAV or its upstream AHU
  {
    VALUES ?ctype {
      brick:Heating_Coil
      brick:Reheat_Valve
      brick:Hot_Water_Valve
      brick:Hot_Water_Circulator_Pump
      brick:Boiler
      brick:Radiator
      brick:Baseboard_Radiator
      brick:Fan_Coil_Unit
      brick:Damper
    }
   

,zone,zone_label,component,component_label
0,urn:example#vav-with-reheat_name-zone_3_7,HVAC_Zone,urn:example#multiple-zone-ahu_ra_damper_3,Damper
1,urn:example#vav-cooling-only_zone_3_1,HVAC_Zone,urn:example#vav-with-reheat_name-dmp_3_9,Damper
2,urn:example#vav-cooling-only_zone_3_10,HVAC_Zone,urn:example#vav-with-reheat_name-dmp_3_9,Damper
3,urn:example#vav-cooling-only_zone_3_11,HVAC_Zone,urn:example#vav-with-reheat_name-dmp_3_9,Damper
4,urn:example#vav-cooling-only_zone_3_12,HVAC_Zone,urn:example#vav-with-reheat_name-dmp_3_9,Damper
...,...,...,...,...
5635,urn:example#vav-with-reheat_name-zone_0_5,HVAC_Zone,urn:example#multiple-zone-ahu_htg_coil_0,Heating_Coil
5636,urn:example#vav-with-reheat_name-zone_0_6,HVAC_Zone,urn:example#multiple-zone-ahu_htg_coil_0,Heating_Coil
5637,urn:example#vav-with-reheat_name-zone_0_7,HVAC_Zone,urn:example#multiple-zone-ahu_htg_coil_0,Heating_Coil
5638,urn:example#vav-with-reheat_name-zone_0_8,HVAC_Zone,urn:example#multiple-zone-ahu_htg_coil_0,Heating_Coil


Displaying File ReAct(w5000)_s223.csv
I'm too cold, find the components most likely at fault.
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>

SELECT DISTINCT ?zone ?zone_temp_prop ?zone_temp ?setpoint_prop ?setpoint ?vav ?damper ?damper_pos_prop ?damper_pos ?heating_coil ?coil_cmd_prop ?coil_cmd
WHERE {
  # Find zone temperature properties
  ?zone_temp_prop qudt:hasQuantityKind quantitykind:Temperature .
  ?zone s223:hasProperty ?zone_temp_prop .
  OPTIONAL { ?zone_temp_prop qudt:value ?zone_temp . }
  OPTIONAL { ?zone_temp_prop s223:hasValue ?zone_temp . }
  OPTIONAL { ?zone_temp_prop ?anyTempPred ?zone_temp . FILTER(isLiteral(?zone_temp)) }

  # Find setpoint for the same zone (if any)
  OPTIONAL {
    ?zone s223:hasProperty ?setpoint_prop .
    ?setpoint_prop s223:hasAspect s223:Aspect-Setpoint .
    OPTIONAL { ?setpoint_prop qudt:value ?setpoint . }
    OPTIONAL { ?setpoint_pr

,ahu,ahu_label,component,component_label
0,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_clg_coil_0,Coil
1,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_htg_coil_0,Coil
2,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_ra_damper_0,Damper
3,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_sa_fan_0,Fan
4,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_clg_coil_1,Coil
5,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_htg_coil_1,Coil
6,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_ra_damper_1,Damper
7,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_sa_fan_1,Fan
8,urn:example#multiple-zone-ahu_name_2,AirHandlingUnit,urn:example#multiple-zone-ahu_clg_coil_2,Coil
9,urn:example#multiple-zone-ahu_name_2,AirHandlingUnit,urn:example#multiple-zone-ahu_htg_coil_2,Coil


What are the discharge air temperature sensors for all VAVs and their associated AHUs?
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>

SELECT DISTINCT ?vav ?dat_sensor ?ahu WHERE {
  # Find all VAVs (SingleDuctTerminal)
  ?vav a s223:SingleDuctTerminal .

  # Find discharge air temperature sensors for the VAV
  {
    ?vav s223:hasProperty ?dat_sensor .
  } UNION {
    ?vav s223:contains ?dat_sensor .
  }
  ?dat_sensor s223:hasQuantityKind quantitykind:Temperature .
  # Accept either hasAspect or hasRole for discharge
  OPTIONAL { ?dat_sensor s223:hasAspect ?aspect . }
  OPTIONAL { ?dat_sensor s223:hasRole ?aspect . }
  FILTER(
    COALESCE(STR(?aspect),"") = "http://data.ashrae.org/standard223#Role-Discharge"
    || COALESCE(STR(?aspect),"") = "Role-Discharge"
    || COALESCE(STR(?aspect),"") = "Discharge"
    || COALESCE(STR(?aspect),"") = "http://data.ashrae.org/standard223#Discharge"
    || !BOUND(?aspect) # Accept if no

In [120]:
summary

[['ReAct(w100)_s223.csv',
  "I'm too cold, find the components most likely at fault.",
  True],
 ['ReAct(w100)_s223.csv',
  'What are all the instances of valve commands for cooling only VAVs?',
  False],
 ['ReAct(w100)_s223.csv',
  'What are all the components of each air handling unit?',
  True],
 ['ReAct(w100)_s223.csv',
  'What are the discharge air temperature sensors for all VAVs and their associated AHUs?',
  False],
 ['lazlo_token_dakgqa_brick.csv',
  'What are the discharge air temperature sensors for all VAVs and their associated AHUs?',
  True],
 ['lazlo_token_dakgqa_brick.csv',
  'What are all the components of each air handling unit?',
  True],
 ['lazlo_token_dakgqa_brick.csv',
  'What are all the instances of valve commands for cooling only VAVs?',
  False],
 ['lazlo_token_dakgqa_brick.csv',
  "I'm too cold, find the components most likely at fault.",
  True],
 ['ReAct(w100)_brick.csv',
  'What are all the instances of valve commands for cooling only VAVs?',
  False],
 ['

In [123]:
# Convert to DataFrame
df = pd.DataFrame(summary, columns=['file', 'question', 'answer'])

# Get last occurrence for each (file, question)
last = df.drop_duplicates(['file', 'question'], keep='last')

# Pivot the table
pivot = last.pivot(index='question', columns='file', values='answer')

# Print markdown table
markdown = pivot.replace({True: "True", False: "False"}).to_markdown()
print(markdown)
pivot.to_csv('query_returns_results.csv')

| question                                                                               | ReAct(w100)_brick.csv   | ReAct(w100)_s223.csv   | ReAct(w5000)_brick.csv   | ReAct(w5000)_s223.csv   | lazlo_token_dakgqa_brick.csv   |
|:---------------------------------------------------------------------------------------|:------------------------|:-----------------------|:-------------------------|:------------------------|:-------------------------------|
| I'm too cold, find the components most likely at fault.                                | False                   | True                   | True                     | False                   | True                           |
| What are all the components of each air handling unit?                                 | True                    | True                   | True                     | True                    | True                           |
| What are all the instances of valve commands for cooling only VAVs?                   

In [53]:
question = "I'm too cold, find the components most likely at fault."
file = "ReAct(w5000)_brick.csv"
for i, query in enumerate(all_queries):
    if question in query['question'] and file in query['source_file']:
        print(query['generated_sparql'])
        display(all_dfs[i])
        print(len(all_dfs[i]))
        break 
# query

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# Replace this with the specific zone or room URI if known, e.g., :vav-cooling-only_zone_1_1
# For demonstration, we select all HVAC_Zone instances as possible affected zones
SELECT DISTINCT ?zone ?zone_label ?component ?component_label WHERE {
  ?zone a brick:HVAC_Zone .
  OPTIONAL { ?zone rdfs:label ?zone_label }

  # Find VAVs or terminal units that serve the zone
  ?vav (brick:feeds|^brick:feeds)* ?zone .
  ?vav a brick:VAV .

  # Find heating-related components that are part of the VAV or its upstream AHU
  {
    VALUES ?ctype {
      brick:Heating_Coil
      brick:Reheat_Valve
      brick:Hot_Water_Valve
      brick:Hot_Water_Circulator_Pump
      brick:Boiler
      brick:Radiator
      brick:Baseboard_Radiator
      brick:Fan_Coil_Unit
      brick:Damper
    }
    ?component a ?ctype .
    # Component is part of the VA

,zone,zoneLabel,equipment,equipmentLabel,point,pointLabel


0


# Checking Specific Queries

In [ ]:
# running fourth s223 Query
query = """PREFIX ns1: <http://data.ashrae.org/standard223#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?component ?componentType ?role
WHERE {
  # Find all air handling units and zone terminals
  ?system a ?systemType .
  FILTER (?systemType IN (ns1:AirHandlingUnit, ns1:SingleDuctTerminal))
  # Get their contained components
  ?system ns1:contains ?component .
  # Identify component type and role
  ?component a ?componentType .
  OPTIONAL { ?component ns1:hasAspect ?role . }
  # Components likely at fault for "too cold":
  FILTER (
    ?componentType IN (ns1:Coil, ns1:Damper, ns1:Fan)
    ||
    EXISTS { ?component ns1:hasAspect ns1:Role-Cooling }
    ||
    EXISTS { ?component ns1:hasAspect ns1:Role-Heating }
  )
}
ORDER BY ?system ?component"""

In [4]:
query_to_df(query,gs)

,component,componentType,role
0,urn:example#multiple-zone-ahu_clg_coil_0,s223:Coil,s223:Role-Cooling
1,urn:example#multiple-zone-ahu_htg_coil_0,s223:Coil,s223:Role-Heating
2,urn:example#multiple-zone-ahu_ra_damper_0,s223:Damper,None
3,urn:example#multiple-zone-ahu_sa_fan_0,s223:Fan,s223:Role-SupplyAir
4,urn:example#multiple-zone-ahu_clg_coil_1,s223:Coil,s223:Role-Cooling
...,...,...,...
191,urn:example#vav-with-reheat_rhc_3_7,s223:Coil,s223:Role-Heating
192,urn:example#vav-with-reheat_dmp_3_8,s223:Damper,None
193,urn:example#vav-with-reheat_rhc_3_8,s223:Coil,s223:Role-Heating
194,urn:example#vav-with-reheat_dmp_3_9,s223:Damper,None


In [35]:
gb = Graph(store = 'Oxigraph')
gb.parse('brick-example-parsed.ttl')
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?component ?component_type ?label ?point ?point_type
WHERE {
  # Find components that are Heating_Coil, Supply_Fan, or Damper
  ?component a ?component_type ;
             rdfs:label ?label .
  FILTER(?component_type IN (
      brick:Heating_Coil,
      brick:Supply_Fan,
      brick:Damper
  ))

  # Optionally get their command/status points
  OPTIONAL {
    ?component brick:hasPoint ?point .
    ?point a ?point_type .
    FILTER(?point_type IN (
      brick:Valve_Position_Command,
      brick:Position_Command,
      brick:Start_Stop_Command,
      brick:Fan_Status,
      brick:Damper_Position_Command
    ))
  }
}
ORDER BY ?component"""

In [36]:
query_to_df(query, gb)

,component,component_type,label,point,point_type
0,urn:example#multiple-zone-ahu_htg_coil_0,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
1,urn:example#multiple-zone-ahu_htg_coil_1,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
2,urn:example#multiple-zone-ahu_htg_coil_2,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
3,urn:example#multiple-zone-ahu_htg_coil_3,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
4,urn:example#multiple-zone-ahu_ra_damper_0,brick:Damper,Damper,urn:example#multiple-zone-ahu_ra_damper-dmppos_0,brick:Damper_Position_Command
...,...,...,...,...,...
191,urn:example#vav-with-reheat_rhc_3_5,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_5,brick:Position_Command
192,urn:example#vav-with-reheat_rhc_3_6,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_6,brick:Position_Command
193,urn:example#vav-with-reheat_rhc_3_7,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_7,brick:Position_Command
194,urn:example#vav-with-reheat_rhc_3_8,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_8,brick:Position_Command


In [53]:
# Running react s223 queries

In [37]:
# preparing parsed 223 example 


In [39]:
#  Brick React 100 queries 
# NO QUERIES RETURNED RESULTS

In [ ]:
brick = Graph(store = 'Oxigraph')
brick.parse('../ontologies/Brick14.ttl', format = 'ttl')
## Error 
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?vav ?valve_command WHERE {
  ?vav a ?vav_type .
  ?vav_type rdfs:subClassOf* brick:VAV .
  ?vav brick:hasPoint ?valve_command .
  ?valve_command a ?cmd_type .
  ?cmd_type rdfs:subClassOf* brick:Valve_Command .
  # Identify cooling-only VAVs by class or property if available
  # Example: ?vav brick:hasMode brick:Cooling_Only .
}"""
query_to_df(query, gb)

,vav,valve_command


In [ ]:
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX : <urn:example#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?ahu ?component WHERE {
  ?ahu a brick:Air_Handling_Unit .
  ?ahu (brick:hasPart|brick:hasSubComponent)+ ?component .
}"""
query_to_df(query, gb)

,ahu,component


In [ ]:
query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?vav ?vav_name ?sensor ?sensor_name ?ahu ?ahu_name WHERE {
  # Find all VAVs
  ?vav a brick:VAV .
  OPTIONAL { ?vav rdfs:label ?vav_name }
  # Find discharge air temperature sensors for each VAV
  ?vav brick:hasPoint ?sensor .
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  OPTIONAL { ?sensor rdfs:label ?sensor_name }
  # Find associated AHU for each VAV
  ?vav brick:isFedBy ?ahu .
  ?ahu a brick:AHU .
  OPTIONAL { ?ahu rdfs:label ?ahu_name }
}"""
query_to_df(query, gb)

,vav,vav_name,sensor,sensor_name,ahu,ahu_name


In [ ]:
# Brick React 5000 queries

In [ ]:
# Fails because it's looking for the valve command on the vav, not on the valve of the coil 
query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?vav ?valve_cmd WHERE {
  ?vav a brick:VAV .
  FILTER NOT EXISTS {
    ?vav brick:hasPart ?heating_coil .
    ?heating_coil a brick:Heating_Coil .
  }
  ?vav brick:hasPoint ?valve_cmd .
  ?valve_cmd a brick:Position_Command .
}"""
query_to_df(query, gb)

,vav,valve_cmd


In [ ]:
# Fails because fo Air Handling Unit alias 
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
SELECT ?ahu ?component WHERE {
  ?ahu a brick:Air_Handling_Unit .
  ?ahu brick:hasPart+ ?component .
}"""
query_to_df(query, gb)

,ahu,component


In [ ]:
# SUCCESS! Also for more specific query 
query = """SELECT ?vav ?ahu ?sensor WHERE {
  ?vav a brick:VAV .
  ?vav brick:hasPoint ?sensor .
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  ?ahu brick:feeds ?vav .
  ?ahu a ?ahu_type .
  FILTER(?ahu_type IN (brick:AHU, brick:Air_Handling_Unit))
}"""
query_to_df(query, gb)

,vav,ahu,sensor
0,urn:example#vav-with-reheat_name_3_9,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_9
1,urn:example#vav-with-reheat_name_3_8,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_8
2,urn:example#vav-with-reheat_name_3_7,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_7
3,urn:example#vav-with-reheat_name_3_6,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_6
4,urn:example#vav-with-reheat_name_3_5,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_5
...,...,...,...
115,urn:example#vav-cooling-only_name_0_12,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_12
116,urn:example#vav-cooling-only_name_0_11,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_11
117,urn:example#vav-cooling-only_name_0_10,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_10
118,urn:example#vav-cooling-only_name_0_1,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_1


In [25]:
# updaing DA KGQA for Brick using openai 4.1

In [ ]:
queries = ["""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?vav ?vavLabel ?sensor ?sensorLabel ?ahu ?ahuLabel
WHERE {
  ?vav a brick:VAV .
  OPTIONAL { ?vav rdfs:label ?vavLabel }
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  OPTIONAL { ?sensor rdfs:label ?sensorLabel }
  ?sensor brick:isPointOf ?vav .
  OPTIONAL {
    ?vav (brick:feeds|^brick:feeds|brick:isFedBy|^brick:isFedBy)+ ?ahu .
    ?ahu a brick:AHU .
    OPTIONAL { ?ahu rdfs:label ?ahuLabel }
  }
}""",
"""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?ahu ?ahuLabel ?component ?componentLabel WHERE {
  ?ahu a brick:Air_Handling_Unit ;
       rdfs:label ?ahuLabel .
  ?ahu ?rel ?component .
  FILTER(?rel IN (brick:Relationship, brick:hasAssociatedTag))
  OPTIONAL { ?component rdfs:label ?componentLabel }
}""",
"""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?valveCommand
WHERE {
  ?valveCommand rdf:type brick:Valve_Command .
}""",
"""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?component ?label ?faultTag WHERE {
  {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Cold .
  } UNION {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Box .
  }
  OPTIONAL { ?component rdfs:label ?label }
  OPTIONAL {
    ?faultTag a brick:Tag ;
      brick:isAssociatedWith ?component .
    FILTER regex(str(?faultTag), "Fault", "i")
  }
}"""]

for query in queries:
    print(query)
    display(query_to_df(query, gb))

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?vav ?vavLabel ?sensor ?sensorLabel ?ahu ?ahuLabel
WHERE {
  ?vav a brick:VAV .
  OPTIONAL { ?vav rdfs:label ?vavLabel }
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  OPTIONAL { ?sensor rdfs:label ?sensorLabel }
  ?sensor brick:isPointOf ?vav .
  OPTIONAL {
    ?vav (brick:feeds|^brick:feeds|brick:isFedBy|^brick:isFedBy)+ ?ahu .
    ?ahu a brick:AHU .
    OPTIONAL { ?ahu rdfs:label ?ahuLabel }
  }
}


,vav,vavLabel,sensor,sensorLabel,ahu,ahuLabel


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?ahu ?ahuLabel ?component ?componentLabel WHERE {
  ?ahu a brick:Air_Handling_Unit ;
       rdfs:label ?ahuLabel .
  ?ahu ?rel ?component .
  FILTER(?rel IN (brick:Relationship, brick:hasAssociatedTag))
  OPTIONAL { ?component rdfs:label ?componentLabel }
}


,ahu,ahuLabel,component,componentLabel


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?valveCommand
WHERE {
  ?valveCommand rdf:type brick:Valve_Command .
}


,valveCommand


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?component ?label ?faultTag WHERE {
  {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Cold .
  } UNION {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Box .
  }
  OPTIONAL { ?component rdfs:label ?label }
  OPTIONAL {
    ?faultTag a brick:Tag ;
      brick:isAssociatedWith ?component .
    FILTER regex(str(?faultTag), "Fault", "i")
  }
}


,component,label,faultTag
0,brick:Cold_Deck,Cold Deck,None
1,brick:Cold_Water_Storage_Tank,Cold Water Storage Tank,None
2,brick:Cold_Box,Cold Box,None
3,brick:Variable_Air_Volume_Box,Variable Air Volume Box,None
4,brick:Variable_Air_Volume_Box_With_Reheat,Variable Air Volume Box With Reheat,None
5,brick:Constant_Air_Volume_Box,Constant Air Volume Box,None
6,brick:Hot_Box,Hot Box,None
7,brick:Hospitality_Box,Hospitality Box,None
8,brick:Environment_Box,Environment Box,None
9,brick:Box_Mode_Command,Box Mode Command,None


In [ ]:
# import pyshacl
# gb_rdflib = Graph()
# gb_rdflib.parse('brick-example.ttl')
# brick = Graph()
# brick.parse('../ontologies/Brick144.ttl')
# v = pyshacl.validate(data_graph = gb_rdflib, shacl_graph=gb_rdflib + brick, ont_graph=brick, inplace=True, inference = 'owlrl')

In [ ]:
# looks like its just because I'm not running reasoning on the model (adding inverse relations)
# I can run reasoning first or just add inverse relatoins maybe 
# Works on reasoned version
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT *
WHERE {
  ?vav a brick:VAV .
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  ?sensor brick:isPointOf ?vav .
  ?vav (brick:feeds|^brick:feeds|brick:isFedBy|^brick:isFedBy)+ ?ahu .
  ?ahu a brick:AHU .
}"""
query_to_df(query, gb_rdflib)

,sensor,ahu,vav
0,urn:example#vav-cooling-only_dat_0_0,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_0
1,urn:example#vav-cooling-only_dat_0_1,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_1
2,urn:example#vav-cooling-only_dat_0_10,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_10
3,urn:example#vav-cooling-only_dat_0_11,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_11
4,urn:example#vav-cooling-only_dat_0_12,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_12
...,...,...,...
115,urn:example#vav-with-reheat_name-dat_3_5,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_5
116,urn:example#vav-with-reheat_name-dat_3_6,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_6
117,urn:example#vav-with-reheat_name-dat_3_7,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_7
118,urn:example#vav-with-reheat_name-dat_3_8,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_8
